In [1]:
import pandas as pd
import numpy as np
import gc

In [2]:
df_allOrdDet = pd.read_pickle('input/all_order_details.p')

In [3]:
df_allOrdDet.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,SucceedingOrdCnt,days_since_first_order,prev_prd,diff_prd,same_prd
0,2539329,1,prior,1,2,8,NaN,"[Soda, Organic-Unsweetened-Vanilla-Almond-Milk...",10.0,NaN,NaN,"[Soda, Organic-Unsweetened-Vanilla-Almond-Milk...",[]
1,2398795,1,prior,2,3,7,15.0,"[Soda, Pistachios, Original-Beef-Jerky, Bag-of...",9.0,15.0,"[Soda, Organic-Unsweetened-Vanilla-Almond-Milk...","[Pistachios, Bag-of-Organic-Bananas, Cinnamon-...","[Soda, Original-Beef-Jerky, Aged-White-Cheddar..."
2,473747,1,prior,3,3,12,21.0,"[Soda, Original-Beef-Jerky, Pistachios, Organi...",8.0,36.0,"[Soda, Pistachios, Original-Beef-Jerky, Bag-of...","[Organic-String-Cheese, Creamy-Almond-Butter]","[Soda, Original-Beef-Jerky, Pistachios]"
3,2254736,1,prior,4,4,7,29.0,"[Soda, Original-Beef-Jerky, Pistachios, Organi...",7.0,65.0,"[Soda, Original-Beef-Jerky, Pistachios, Organi...",[XL-Pick-A-Size-Paper-Towel-Rolls],"[Soda, Original-Beef-Jerky, Pistachios, Organi..."
4,431534,1,prior,5,4,15,28.0,"[Soda, Original-Beef-Jerky, Pistachios, Organi...",6.0,93.0,"[Soda, Original-Beef-Jerky, Pistachios, Organi...","[Organic-Fuji-Apples, Honeycrisp-Apples, Bartl...","[Soda, Original-Beef-Jerky, Pistachios, Organi..."


In [4]:
df_allOrdDet=df_allOrdDet[df_allOrdDet['eval_set']!='prior']

In [5]:
df_allOrdDet['eval_set'].unique()

array(['train', 'test'], dtype=object)

In [6]:
df_allOrdDet = df_allOrdDet[['order_id','user_id','order_number','SucceedingOrdCnt','eval_set']]

In [7]:
df_allOrdDet.sort_values(['user_id','order_number','order_id'],inplace=True)

In [8]:
label = pd.read_pickle('input/label.p')[['order_id']]

In [9]:
df_allOrdDet=df_allOrdDet[~df_allOrdDet['order_id'].isin(label['order_id'])]

In [10]:
del label

In [11]:
df_allOrdDet=df_allOrdDet.merge(pd.read_pickle('input/all_data.p')[['order_id','product_id','order_hour_of_day','order_dow','reordered']],on='order_id',how='left')

In [12]:
df_allOrdDet=df_allOrdDet.merge(pd.read_pickle('input/product_detail.p')[['product_id','aisle_id','department_id']],on='product_id',how='left')

In [13]:
df_allOrdDet.head()

,order_id,user_id,order_number,SucceedingOrdCnt,eval_set,product_id,order_hour_of_day,order_dow,reordered,aisle_id,department_id
0,1187899,1,11,0.0,train,196.0,8.0,4.0,1.0,77.0,7.0
1,1187899,1,11,0.0,train,25133.0,8.0,4.0,1.0,21.0,16.0
2,1187899,1,11,0.0,train,38928.0,8.0,4.0,1.0,120.0,16.0
3,1187899,1,11,0.0,train,26405.0,8.0,4.0,1.0,54.0,17.0
4,1187899,1,11,0.0,train,39657.0,8.0,4.0,1.0,45.0,19.0


In [14]:
df_allOrdDet['eval_set'].unique()

array(['train', 'test'], dtype=object)

In [15]:
df_allOrdDet.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1459617 entries, 0 to 1459616
Data columns (total 11 columns):
order_id             1459617 non-null int64
user_id              1459617 non-null int64
order_number         1459617 non-null int64
SucceedingOrdCnt     1384617 non-null float64
eval_set             1459617 non-null object
product_id           1384617 non-null float64
order_hour_of_day    1384617 non-null float64
order_dow            1384617 non-null float64
reordered            1384617 non-null float64
aisle_id             1384617 non-null float64
department_id        1384617 non-null float64
dtypes: float64(7), int64(3), object(1)
memory usage: 208.7 MB


In [16]:
# Collect Order ids of succeeding 3 orders
for i in range(1,4):
    df_allOrdDet['t-{}_order_id'.format(i)] = df_allOrdDet.groupby('user_id')['order_id'].shift(i)

In [17]:
df_allOrdDet.isnull().sum()

order_id                  0
user_id                   0
order_number              0
SucceedingOrdCnt      75000
eval_set                  0
product_id            75000
order_hour_of_day     75000
order_dow             75000
reordered             75000
aisle_id              75000
department_id         75000
t-1_order_id         206209
t-2_order_id         330573
t-3_order_id         447569
dtype: int64

In [18]:
df_allOrdDet.dropna(inplace=True)

In [19]:
df_allOrdDet['SucceedingOrdCnt'].fillna(value=0,inplace=True)

In [20]:
df_allOrdDet['SucceedingOrdCnt']=df_allOrdDet['SucceedingOrdCnt'].astype(int)

In [21]:
col = [c for c in df_allOrdDet.columns if 'order_id' in c]

In [22]:
for c in col:
    df_allOrdDet[c] = df_allOrdDet[c].map(int)

In [23]:
df_allOrdDet.reset_index(drop=1, inplace=True)

In [24]:
df_allOrdDet.head()

,order_id,user_id,order_number,SucceedingOrdCnt,eval_set,product_id,order_hour_of_day,order_dow,reordered,aisle_id,department_id,t-1_order_id,t-2_order_id,t-3_order_id
0,1187899,1,11,0,train,26405.0,8.0,4.0,1.0,54.0,17.0,1187899,1187899,1187899
1,1187899,1,11,0,train,39657.0,8.0,4.0,1.0,45.0,19.0,1187899,1187899,1187899
2,1187899,1,11,0,train,10258.0,8.0,4.0,1.0,117.0,19.0,1187899,1187899,1187899
3,1187899,1,11,0,train,13032.0,8.0,4.0,1.0,121.0,14.0,1187899,1187899,1187899
4,1187899,1,11,0,train,26088.0,8.0,4.0,1.0,23.0,19.0,1187899,1187899,1187899


In [25]:
df_allOrdDet['eval_set'].unique()

array(['train'], dtype=object)

## Combining Engineered Features
____________

#### Combining User Behavior Features

In [26]:
df_userBehavior = pd.read_pickle('input/userOrdSize.p')

In [27]:
df_userBehavior.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 6 columns):
user_id                   206209 non-null int64
user_order_size-min       206209 non-null int64
user_order_size-max       206209 non-null int64
user_order_size-median    206209 non-null float64
user_order_size-mean      206209 non-null float64
user_order_size-std       206209 non-null float64
dtypes: float64(3), int64(3)
memory usage: 9.4 MB


In [28]:
#Downcast column datatypes to save memory
df_userBehavior=df_userBehavior.apply(pd.to_numeric,downcast='unsigned')
df_userBehavior.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 6 columns):
user_id                   206209 non-null uint32
user_order_size-min       206209 non-null uint8
user_order_size-max       206209 non-null uint8
user_order_size-median    206209 non-null float64
user_order_size-mean      206209 non-null float64
user_order_size-std       206209 non-null float64
dtypes: float64(3), uint32(1), uint8(2)
memory usage: 5.9 MB


In [29]:
df_userBehavior.head()

,user_id,user_order_size-min,user_order_size-max,user_order_size-median,user_order_size-mean,user_order_size-std
0,1,4,9,5.5,5.900000,1.523884
1,2,5,26,13.5,13.928571,5.717238
2,3,5,11,8.0,7.454545,2.161649
3,4,2,7,3.0,3.750000,2.362908
4,5,5,12,10.0,9.250000,3.095696


In [30]:
df_allOrdDet = df_allOrdDet.merge(df_userBehavior, on='user_id',how='left')

In [31]:
df_allOrdDet.head()

,order_id,user_id,order_number,SucceedingOrdCnt,eval_set,product_id,order_hour_of_day,order_dow,reordered,aisle_id,department_id,t-1_order_id,t-2_order_id,t-3_order_id,user_order_size-min,user_order_size-max,user_order_size-median,user_order_size-mean,user_order_size-std
0,1187899,1,11,0,train,26405.0,8.0,4.0,1.0,54.0,17.0,1187899,1187899,1187899,4,9,5.5,5.9,1.523884
1,1187899,1,11,0,train,39657.0,8.0,4.0,1.0,45.0,19.0,1187899,1187899,1187899,4,9,5.5,5.9,1.523884
2,1187899,1,11,0,train,10258.0,8.0,4.0,1.0,117.0,19.0,1187899,1187899,1187899,4,9,5.5,5.9,1.523884
3,1187899,1,11,0,train,13032.0,8.0,4.0,1.0,121.0,14.0,1187899,1187899,1187899,4,9,5.5,5.9,1.523884
4,1187899,1,11,0,train,26088.0,8.0,4.0,1.0,23.0,19.0,1187899,1187899,1187899,4,9,5.5,5.9,1.523884


#### Combining Item Characteristics Features

In [32]:
df_itemChar = pd.read_pickle('input/hrGrp.p')
df_itemChar.head()

,product_id,order_hour_of_day,item_hour_cnt,itemHrDist
0,1,0,12,0.006224
1,1,1,12,0.006224
2,1,2,9,0.004668
3,1,3,5,0.002593
4,1,4,4,0.002075


In [33]:
pd.read_pickle('input/hrGrpUnq.p').head()

,product_id,order_hour_of_day,unique_item_hour_cnt,unqItemHrDist
0,1,0,11,0.007412
1,1,1,10,0.006739
2,1,2,9,0.006065
3,1,3,5,0.003369
4,1,4,3,0.002022


In [34]:
df_itemChar = df_itemChar.merge(pd.read_pickle('input/hrGrpUnq.p'),on=['product_id','order_hour_of_day'],how='left')

In [35]:
df_itemChar=df_itemChar.apply(pd.to_numeric,downcast='unsigned')
df_itemChar.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 777214 entries, 0 to 777213
Data columns (total 6 columns):
product_id              777214 non-null uint16
order_hour_of_day       777214 non-null uint8
item_hour_cnt           777214 non-null uint16
itemHrDist              777214 non-null float64
unique_item_hour_cnt    777214 non-null uint16
unqItemHrDist           777214 non-null float64
dtypes: float64(2), uint16(3), uint8(1)
memory usage: 23.0 MB


In [36]:
df_allOrdDet = df_allOrdDet.merge(df_itemChar,on=['product_id','order_hour_of_day'],how='left')

In [37]:
df_itemChar2 = pd.read_pickle('input/dowGrp.p')

In [38]:
df_itemChar2 = df_itemChar2.merge(pd.read_pickle('input/unqDowGrp.p'),on=['product_id','order_dow'],how='left')

In [39]:
df_itemChar2=df_itemChar2.apply(pd.to_numeric,downcast='unsigned')
df_itemChar2.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 319956 entries, 0 to 319955
Data columns (total 6 columns):
product_id             319956 non-null uint16
order_dow              319956 non-null uint8
item_dow_cnt           319956 non-null uint32
itemDowDist            319956 non-null float64
unique_item_dow_cnt    319956 non-null uint16
unqItemDowDist         319956 non-null float64
dtypes: float64(2), uint16(2), uint32(1), uint8(1)
memory usage: 10.1 MB


In [40]:
df_allOrdDet = df_allOrdDet.merge(df_itemChar2,on=['product_id','order_dow'],how='left')

In [41]:
df_allOrdDet = df_allOrdDet.merge(pd.read_pickle('input/prodComb.p'),on=['product_id'],how='left')

In [42]:
df_allOrdDet.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1012048 entries, 0 to 1012047
Data columns (total 31 columns):
order_id                  1012048 non-null int64
user_id                   1012048 non-null int64
order_number              1012048 non-null int64
SucceedingOrdCnt          1012048 non-null int32
eval_set                  1012048 non-null object
product_id                1012048 non-null float64
order_hour_of_day         1012048 non-null float64
order_dow                 1012048 non-null float64
reordered                 1012048 non-null float64
aisle_id                  1012048 non-null float64
department_id             1012048 non-null float64
t-1_order_id              1012048 non-null int64
t-2_order_id              1012048 non-null int64
t-3_order_id              1012048 non-null int64
user_order_size-min       1012048 non-null uint8
user_order_size-max       1012048 non-null uint8
user_order_size-median    1012048 non-null float64
user_order_size-mean      1012048 non-n

In [43]:
df_allOrdDet.head()

,order_id,user_id,order_number,SucceedingOrdCnt,eval_set,product_id,order_hour_of_day,order_dow,reordered,aisle_id,...,unique_item_hour_cnt,unqItemHrDist,item_dow_cnt,itemDowDist,unique_item_dow_cnt,unqItemDowDist,prodComb_mean,prodComb_min,prodComb_max,prodComb_std
0,1187899,1,11,0,train,26405.0,8.0,4.0,1.0,54.0,...,76,0.064461,193,0.146323,151,0.137273,4.613333,1.0,28.0,3.662998
1,1187899,1,11,0,train,39657.0,8.0,4.0,1.0,45.0,...,191,0.055059,707,0.133952,434,0.158279,7.503308,1.0,52.0,5.384723
2,1187899,1,11,0,train,10258.0,8.0,4.0,1.0,117.0,...,92,0.064201,288,0.140488,174,0.150780,6.897539,1.0,39.0,4.879103
3,1187899,1,11,0,train,13032.0,8.0,4.0,1.0,121.0,...,187,0.063888,556,0.142418,352,0.143673,9.492694,1.0,100.0,7.732432
4,1187899,1,11,0,train,26088.0,8.0,4.0,1.0,23.0,...,108,0.047141,346,0.130124,263,0.131369,10.364286,1.0,59.0,7.477187


#### Combining User Product Interaction Features

In [44]:
df_totalBuys=pd.read_pickle('input/totalbuys.p')

In [45]:
df_totalBuys=df_totalBuys.apply(pd.to_numeric,downcast='unsigned')
df_totalBuys.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12084910 entries, 0 to 12084909
Data columns (total 5 columns):
user_id            uint32
order_id           uint32
product_id         uint16
total_buy          uint8
total_buy_ratio    float64
dtypes: float64(1), uint16(1), uint32(2), uint8(1)
memory usage: 311.2 MB


In [46]:
df_allOrdDet = df_allOrdDet.merge(df_totalBuys,on=['user_id','product_id'],how='left')

In [47]:
prdOrdPoss=pd.read_pickle('input/prdOrdPoss.p')

In [48]:
prdOrdPoss=prdOrdPoss.apply(pd.to_numeric,downcast='unsigned')
prdOrdPoss.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12084910 entries, 0 to 12084909
Data columns (total 7 columns):
user_id        uint32
product_id     uint16
count          uint8
minPrdOrd      uint8
maxOrd         uint8
possibility    uint8
ordPoss        float64
dtypes: float64(1), uint16(1), uint32(1), uint8(4)
memory usage: 299.7 MB


In [49]:
df_allOrdDet = df_allOrdDet.merge(prdOrdPoss,on=['user_id','product_id'],how='left')

In [50]:
df_allOrdDet.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1012048 entries, 0 to 1012047
Data columns (total 39 columns):
order_id_x                1012048 non-null int64
user_id                   1012048 non-null int64
order_number              1012048 non-null int64
SucceedingOrdCnt          1012048 non-null int32
eval_set                  1012048 non-null object
product_id                1012048 non-null float64
order_hour_of_day         1012048 non-null float64
order_dow                 1012048 non-null float64
reordered                 1012048 non-null float64
aisle_id                  1012048 non-null float64
department_id             1012048 non-null float64
t-1_order_id              1012048 non-null int64
t-2_order_id              1012048 non-null int64
t-3_order_id              1012048 non-null int64
user_order_size-min       1012048 non-null uint8
user_order_size-max       1012048 non-null uint8
user_order_size-median    1012048 non-null float64
user_order_size-mean      1012048 non-n

In [51]:
df_allOrdDet=df_allOrdDet.drop(['order_id_y'],axis=1)
df_allOrdDet.rename(columns={'order_id_x':'order_id'},inplace=True)

In [52]:
df_allOrdDet.head()

,order_id,user_id,order_number,SucceedingOrdCnt,eval_set,product_id,order_hour_of_day,order_dow,reordered,aisle_id,...,prodComb_min,prodComb_max,prodComb_std,total_buy,total_buy_ratio,count,minPrdOrd,maxOrd,possibility,ordPoss
0,1187899,1,11,0,train,26405.0,8.0,4.0,1.0,54.0,...,1.0,28.0,3.662998,2.0,inf,2.0,1.0,9.0,9.0,0.222222
1,1187899,1,11,0,train,39657.0,8.0,4.0,1.0,45.0,...,1.0,52.0,5.384723,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1187899,1,11,0,train,10258.0,8.0,4.0,1.0,117.0,...,1.0,39.0,4.879103,8.0,0.888889,8.0,2.0,9.0,8.0,1.000000
3,1187899,1,11,0,train,13032.0,8.0,4.0,1.0,121.0,...,1.0,100.0,7.732432,2.0,0.222222,2.0,2.0,9.0,8.0,0.250000
4,1187899,1,11,0,train,26088.0,8.0,4.0,1.0,23.0,...,1.0,59.0,7.477187,2.0,inf,2.0,1.0,9.0,9.0,0.222222


#### Combining Day and Hour Features

In [53]:
df_dowOrdTrd = pd.read_pickle('input/dowOrdTrd.p')

In [54]:
df_dowOrdTrd=df_dowOrdTrd.apply(pd.to_numeric,downcast='unsigned')
df_dowOrdTrd.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 4 columns):
order_dow          7 non-null uint8
dow_order_count    7 non-null float64
dow_item_cnt       7 non-null float64
dow_rank_diff      7 non-null float64
dtypes: float64(3), uint8(1)
memory usage: 255.0 bytes


In [55]:
df_allOrdDet = df_allOrdDet.merge(df_dowOrdTrd,on='order_dow',how='left')

In [56]:
df_hourOrdTrd=pd.read_pickle('input/hourOrdTrd.p').reset_index()

In [57]:
df_hourOrdTrd=df_hourOrdTrd.apply(pd.to_numeric,downcast='unsigned')
df_hourOrdTrd.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 4 columns):
order_hour_of_day    24 non-null uint8
hour_order_count     24 non-null float64
hour_item_count      24 non-null float64
hour_rank_diff       24 non-null float64
dtypes: float64(3), uint8(1)
memory usage: 680.0 bytes


In [58]:
df_allOrdDet = df_allOrdDet.merge(df_hourOrdTrd,on='order_hour_of_day',how='left')

### Export All Engineered Features

In [59]:
df_allOrdDet.head()

,order_id,user_id,order_number,SucceedingOrdCnt,eval_set,product_id,order_hour_of_day,order_dow,reordered,aisle_id,...,minPrdOrd,maxOrd,possibility,ordPoss,dow_order_count,dow_item_cnt,dow_rank_diff,hour_order_count,hour_item_count,hour_rank_diff
0,1187899,1,11,0,train,26405.0,8.0,4.0,1.0,54.0,...,1.0,9.0,9.0,0.222222,0.125184,0.117113,0.0,0.052756,0.053516,-1.0
1,1187899,1,11,0,train,39657.0,8.0,4.0,1.0,45.0,...,NaN,NaN,NaN,NaN,0.125184,0.117113,0.0,0.052756,0.053516,-1.0
2,1187899,1,11,0,train,10258.0,8.0,4.0,1.0,117.0,...,2.0,9.0,8.0,1.000000,0.125184,0.117113,0.0,0.052756,0.053516,-1.0
3,1187899,1,11,0,train,13032.0,8.0,4.0,1.0,121.0,...,2.0,9.0,8.0,0.250000,0.125184,0.117113,0.0,0.052756,0.053516,-1.0
4,1187899,1,11,0,train,26088.0,8.0,4.0,1.0,23.0,...,1.0,9.0,9.0,0.222222,0.125184,0.117113,0.0,0.052756,0.053516,-1.0


In [60]:
df_allOrdDet.isnull().sum()

order_id                       0
user_id                        0
order_number                   0
SucceedingOrdCnt               0
eval_set                       0
product_id                     0
order_hour_of_day              0
order_dow                      0
reordered                      0
aisle_id                       0
department_id                  0
t-1_order_id                   0
t-2_order_id                   0
t-3_order_id                   0
user_order_size-min            0
user_order_size-max            0
user_order_size-median         0
user_order_size-mean           0
user_order_size-std            0
item_hour_cnt                  0
itemHrDist                     0
unique_item_hour_cnt           0
unqItemHrDist                  0
item_dow_cnt                   0
itemDowDist                    0
unique_item_dow_cnt            0
unqItemDowDist                 0
prodComb_mean                 42
prodComb_min                  42
prodComb_max                  42
prodComb_s

In [ ]:
df_allOrdDet.fillna(value=0,inplace=True)

In [ ]:
len(df_allOrdDet.index)

In [ ]:
df_allOrdDet_train=df_allOrdDet[df_allOrdDet['eval_set']=='train']

In [ ]:
df_allOrdDet_train.to_pickle('input/all_Eng_Features_train.p')

In [ ]:
df_allOrdDet['reordered'].describe()

In [ ]:
df_allOrdDet['eval_set'].unique()